# Miyagi Conversation use case using Langchain and Qdrant

Qdrant implemented the [fastest vector index algorithm](https://qdrant.tech/documentation/indexing/#vector-index) (as of '23), HNSW, and SK is also planning to leverage Qdrant. So this will be our first choice for vector store with Miyagi use cases.


In [ ]:
! pip install langchain qdrant-client openai cohere
! pip install sentence-transformers
! pip install wget

In [43]:
import json

# loaded some random questions from
# https://pillarwm.com/10-questions-to-ask-your-wealth-manager-and-the-answers-you-want/
with open("user-questions.json", "r") as fp:
    questions = json.load(fp)

with open("miyagi-answers.json", "r") as fp:
    answers = json.load(fp)

questions[0] + ": " + answers[0]

'What is your minimum asset requirement?: $5 million or more.'

In [50]:
import os
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_BASE"] = "https://reddog-openai.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = "<AZ_OPENAI_KEY>"

In [59]:
from langchain.embeddings import OpenAIEmbeddings

In [60]:
embeddings = OpenAIEmbeddings()

In [ ]:
! docker run -d -p "6333:6333" -p "6334:6334" qdrant/qdrant:latest

In [61]:
from langchain.vectorstores import Qdrant

In [62]:
doc_store = Qdrant.from_texts(
    answers, embeddings, host="localhost" 
)

In [63]:
doc_store.similarity_search(questions[0])

[Document(page_content='Use strategic management, charge no commissions or hidden fees, disclose everything, and minimize taxes.', lookup_str='', metadata={}, lookup_index=0),
 Document(page_content='$5 million or more.', lookup_str='', metadata={}, lookup_index=0),
 Document(page_content='Close to the wealth managers years of experience. If he has 20 years of experience, look for clients whove been with him for over ten years on average. If 30 years of experience, then over 20 years on average.', lookup_str='', metadata={}, lookup_index=0),
 Document(page_content='At least 10 years. Answers may vary.', lookup_str='', metadata={}, lookup_index=0)]

In [64]:
from langchain.llms import OpenAI
from langchain.chains import VectorDBQA

In [65]:
llm = OpenAI() # OpenAI API key is read from env variables

In [66]:
qa = VectorDBQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    vectorstore=doc_store,
    return_source_documents=False,
)

In [68]:
import random

random.seed(42)
selected_questions = random.choices(questions, k=5)

In [69]:
for question in selected_questions:
    print(">", question)
    print(qa.run(question), end="\n\n")

> What do you do to minimize my costs?
 Use strategic management, charge no commissions or hidden fees, disclose everything, and minimize taxes.

> What is your minimum asset requirement?
 My minimum asset requirement is $5 million or more.

> How long do your clients stay with you on average?
 At least 10 years.

> How long have you been a wealth manager?
 I don't know.

> What do you do to minimize my costs?
 Use strategic management, charge no commissions or hidden fees, disclose everything, and minimize taxes.



In [70]:
from langchain.chains import ChatVectorDBChain

In [71]:
chat_qa = ChatVectorDBChain.from_llm(
    llm=llm,
    vectorstore=doc_store,
)

In [72]:
chat_history = []
while True:
    query = input("> Question: ").strip()
    if len(query) == 0:
        break
    result = chat_qa({"question": query, "chat_history": chat_history})
    chat_history.append((query, result["answer"]))
    print(result["answer"], end="\n\n")

 You can use strategic management, charge no commissions or hidden fees, disclose everything, and minimize taxes.

